<a href="https://colab.research.google.com/github/Kensuzuki95/Corporate_AI_Ethics_Guideline_Analysis/blob/main/Document_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** **
# Step 1: Load Package
** **

In [2]:
import numpy as np 
import pandas as pd 
import requests
import io

** **
# Step 2: Load Data
** **

In [3]:
# Downloading the csv file from your GitHub account

url_1 = ("https://raw.githubusercontent.com/Kensuzuki95/Corporate_AI_Ethics_Guideline_Analysis/main/Dataset/Dataset_Filtered.csv")
download = requests.get(url_1).content

dataset = pd.read_csv(io.StringIO(download.decode('utf-8')))

dataset.head()

,No.,Company Name,Country,Industry,Published Year,Last Revised,Link,Document Name,Main Text,Comment
0,1,Accenture,Ireland,Consulting,03-30-2021,03-30-2021,https://www.accenture.com/content/dam/accentur...,Responsible AI From principles to practice,Responsible AI\r\nFrom principles to practice\...,Addtional Details: https://www.accenture.com/u...
1,2,Adobe,United States of America,Software,NaN,NaN,https://www.adobe.com/content/dam/cc/en/ai-eth...,Adobe’s Commitment to AI Ethics,"Adobe’s Commitment to AI Ethics\r\nAt Adobe, o...",Addtional Details: https://www.adobe.com/conte...
2,3,Alphabet,United States of America,Software,NaN,NaN,https://ai.google/responsibilities/responsible...,Responsible AI practices,Responsible AI practices\r\nThe development of...,Addtional Information: https://ai.google/princ...
3,4,Amazon,United States of America,Software,NaN,NaN,https://d1.awsstatic.com/responsible-machine-l...,Responsible Use of Machine Learning,"Responsible Use of Machine Learning\r\nAt AWS,...",NaN
4,5,Atos,France,Consulting,NaN,NaN,https://atos.net/en/lp/cybersecurity-magazine-...,The Atos Blueprint for Responsible AI,AI is a broad topic encompassing many differen...,NaN


## Clean the Dataset Format

In [4]:
#Check for unecesarry columns
dataset.columns

Index(['No.', 'Company Name', 'Country', 'Industry', 'Published Year',
       'Last Revised', 'Link', 'Document Name', 'Main Text', 'Comment'],
      dtype='object')

In [5]:
text_data = dataset.drop(columns=['No.','Country', 'Industry', 'Published Year', 'Last Revised', 'Link', 'Comment'], axis=1)
text_data.info()
text_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company Name   49 non-null     object
 1   Document Name  49 non-null     object
 2   Main Text      49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


,Company Name,Document Name,Main Text
0,Accenture,Responsible AI From principles to practice,Responsible AI\r\nFrom principles to practice\...
1,Adobe,Adobe’s Commitment to AI Ethics,"Adobe’s Commitment to AI Ethics\r\nAt Adobe, o..."
2,Alphabet,Responsible AI practices,Responsible AI practices\r\nThe development of...
3,Amazon,Responsible Use of Machine Learning,"Responsible Use of Machine Learning\r\nAt AWS,..."
4,Atos,The Atos Blueprint for Responsible AI,AI is a broad topic encompassing many differen...


** **
#Step 3: Data Cleaning
** **

Since the goal of this analysis is to perform topic modeling, we will solely focus on the text data from each paper, and drop other metadata columns

## Remove punctuation/lower casing

Next, let’s perform a simple preprocessing on the content of paper_text column to make them more amenable for analysis, and reliable results. To do that, we’ll use a regular expression to remove any punctuation, and then lowercase the text

In [5]:
# Load the regular expression library
import re
import nltk


# Remove punctuation
text_data['Main_Text_Processed'] = text_data['Main Text'].map(lambda x: re.sub('[,\.!?()]', '', x))

# Convert the text to lowercase
text_data['Main_Text_Processed'] = text_data['Main_Text_Processed'].map(lambda x: x.lower())

# Applying Tokenization
nltk.download('punkt')
text_data['Main_Text_Tokenized'] = text_data.apply(lambda row: nltk.word_tokenize(row['Main_Text_Processed']), axis=1)

# Print out the first rows of papers
#training_data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Tokenize words and further clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [6]:
text_data.head()

,Company Name,Document Name,Main Text,Main_Text_Processed,Main_Text_Tokenized
0,Accenture,Responsible AI From principles to practice,Responsible AI\r\nFrom principles to practice\...,responsible ai\r\nfrom principles to practice\...,"[responsible, ai, from, principles, to, practi..."
1,Adobe,Adobe’s Commitment to AI Ethics,"Adobe’s Commitment to AI Ethics\r\nAt Adobe, o...",adobe’s commitment to ai ethics\r\nat adobe ou...,"[adobe, ’, s, commitment, to, ai, ethics, at, ..."
2,Alphabet,Responsible AI practices,Responsible AI practices\r\nThe development of...,responsible ai practices\r\nthe development of...,"[responsible, ai, practices, the, development,..."
3,Amazon,Responsible Use of Machine Learning,"Responsible Use of Machine Learning\r\nAt AWS,...",responsible use of machine learning\r\nat aws ...,"[responsible, use, of, machine, learning, at, ..."
4,Atos,The Atos Blueprint for Responsible AI,AI is a broad topic encompassing many differen...,ai is a broad topic encompassing many differen...,"[ai, is, a, broad, topic, encompassing, many, ..."


In [7]:
#defining the function to remove stopwords from tokenized text
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

#applying the function
text_data['Main_text_without_stopwords'] = text_data['Main_Text_Tokenized'].apply(lambda x:remove_stopwords(x))
#training_data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer

#defining the object for stemming
porter_stemmer = PorterStemmer()

#defining a function for stemming
def stemming(text):
  stem_text = [porter_stemmer.stem(word) for word in text]
  return stem_text

text_data['Main_text_stemmed'] = text_data['Main_text_without_stopwords'].apply(lambda x: stemming(x))
text_data.head()

,Company Name,Document Name,Main Text,Main_Text_Processed,Main_Text_Tokenized,Main_text_without_stopwords,Main_text_stemmed
0,Accenture,Responsible AI From principles to practice,Responsible AI\r\nFrom principles to practice\...,responsible ai\r\nfrom principles to practice\...,"[responsible, ai, from, principles, to, practi...","[responsible, ai, principles, practice, conten...","[respons, ai, principl, practic, content, resp..."
1,Adobe,Adobe’s Commitment to AI Ethics,"Adobe’s Commitment to AI Ethics\r\nAt Adobe, o...",adobe’s commitment to ai ethics\r\nat adobe ou...,"[adobe, ’, s, commitment, to, ai, ethics, at, ...","[adobe, ’, commitment, ai, ethics, adobe, purp...","[adob, ’, commit, ai, ethic, adob, purpos, ser..."
2,Alphabet,Responsible AI practices,Responsible AI practices\r\nThe development of...,responsible ai practices\r\nthe development of...,"[responsible, ai, practices, the, development,...","[responsible, ai, practices, development, ai, ...","[respons, ai, practic, develop, ai, creat, new..."
3,Amazon,Responsible Use of Machine Learning,"Responsible Use of Machine Learning\r\nAt AWS,...",responsible use of machine learning\r\nat aws ...,"[responsible, use, of, machine, learning, at, ...","[responsible, use, machine, learning, aws, pro...","[respons, use, machin, learn, aw, proud, suppo..."
4,Atos,The Atos Blueprint for Responsible AI,AI is a broad topic encompassing many differen...,ai is a broad topic encompassing many differen...,"[ai, is, a, broad, topic, encompassing, many, ...","[ai, broad, topic, encompassing, many, differe...","[ai, broad, topic, encompass, mani, differ, fa..."


** **
#Step 4: Measure Text Similarity
** **



## Create BERT-based Text Similaarity Scoring Model

In [6]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=067ae26e37dcc8985f253f91cfa53b0c56124bb0656ca77f7dab535d59db4bab
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [52]:
# Create a Dataset with only Company Name & Main Text of the AI Ethics Principle Document
similarity_data = text_data[['Company Name','Main Text']]
sentences = similarity_data['Main Text'].values.tolist()

In [50]:
# Load UNESCO's AI Ethics Principles Dataset
url_2 = ("https://raw.githubusercontent.com/Kensuzuki95/Corporate_AI_Ethics_Guideline_Analysis/main/Dataset/UNESCO_AI_Ethics_Principles.csv")
download = requests.get(url_2).content

principles = pd.read_csv(io.StringIO(download.decode('utf-8')))
principles.head()

,No.,Principle Name,Content
0,1,Proportionality and Do No Harm,It should be recognized that AI technologies d...
1,2,Safety and security,"Unwanted harms (safety risks), as well as vuln..."
2,3,Fairness and non-discrimination,AI actors should promote social justice and sa...
3,4,Sustainability,The development of sustainable societies relie...
4,5,"Right to Privacy, and Data Protection","Privacy, a right essential to the protection o..."


In [59]:
## Create a function
from sklearn.metrics.pairwise import cosine_similarity

def similarity_score(principle_number):
  principle_principle_number = principles.iloc[principle_number-1]['Content']
  sentences_principle_number = sentences.copy()
  sentences_principle_number.insert(0, principle_principle_number) 
  sentence_embeddings_principle_number = model.encode(sentences_principle_number)
  sentence_embeddings_principle_number.shape
  results_principle_number = cosine_similarity([sentence_embeddings_principle_number[0]], 
                                               sentence_embeddings_principle_number[1:])
  results_principle_number = results_principle_number.tolist()
  results_principle_number = results_principle_number[0]
  print("UNESCO AI Ethics Princple #" + str(principle_number) + ":" + '\n' + str(principle_principle_number))
  #(len(results_principle_number))
  return results_principle_number;

## Principle 1

In [63]:
# Product Similarity Score of all documents to Principle #1
results_1 = similarity_score(principle_number = 1)
#results_1

UNESCO AI Ethics Princple #1:
It should be recognized that AI technologies do not necessarily, per se, ensure human and environmental and ecosystem flourishing. Furthermore, none of the processes related to the AI system life cycle shall exceed what is necessary to achieve legitimate aims or objectives and should be appropriate to the context. In the event of possible occurrence of any harm to human beings, human rights and fundamental freedoms, communities and society at large or the environment and ecosystems, the implementation of procedures for risk assessment and the adoption of measures in order to preclude the occurrence of such harm should be ensured.
The choice to use AI systems and which AI method to use should be justified in the following ways: (a) the AI method chosen should be appropriate and proportional to achieve a given legitimate aim; (b) the AI method chosen should not infringe upon the foundational values captured in this document, in particular, its use must not v

In [64]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_1'] = results_1
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 2

In [65]:
# Product Similarity Score of all documents to Principle #2
results_2 = similarity_score(principle_number = 2)
#results_2

UNESCO AI Ethics Princple #2:
Unwanted harms (safety risks), as well as vulnerabilities to attack (security risks) should be avoided and should be addressed, prevented and eliminated throughout the life cycle of AI systems to ensure human, environmental and ecosystem safety and security. Safe and secure AI will be enabled by the development of sustainable, privacyprotective data access frameworks that foster better training and validation of AI models utilizing quality data.


In [66]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_2'] = results_2
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 3

In [67]:
# Product Similarity Score of all documents to Principle #3
results_3 = similarity_score(principle_number = 3)
#results_3

UNESCO AI Ethics Princple #3:
AI actors should promote social justice and safeguard fairness and non-discrimination of any kind in compliance with international law. This implies an inclusive approach to ensuring that the benefits of AI technologies are available and accessible to all, taking into consideration the specific needs of different age groups, cultural systems, different language groups, persons with disabilities, girls and women, and disadvantaged, marginalized and vulnerable people or people in vulnerable situations. Member States should work to promote inclusive access for all, including local communities, to AI systems with locally relevant content and services, and with respect for multilingualism and cultural diversity. Member States should work to tackle digital divides and ensure inclusive access to and participation in the development of AI. At the national level, Member States should promote equity between rural and urban areas, and among all persons regardless of 

In [68]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_3'] = results_3
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 4

In [69]:
# Product Similarity Score of all documents to Principle #4
results_4 = similarity_score(principle_number = 4)
#results_4

UNESCO AI Ethics Princple #4:
The development of sustainable societies relies on the achievement of a complex set of objectives on a continuum of human, social, cultural, economic and environmental dimensions. The advent of AI technologies can either benefit sustainability objectives or hinder their realization, depending on how they are applied across countries with varying levels of development. The continuous assessment of the human, social, cultural, economic and environmental impact of AI technologies should therefore be carried out with full cognizance of the implications of AI technologies for sustainability as a set of constantly evolving goals across a range of dimensions, such as currently identified in the Sustainable Development Goals (SDGs) of the United Nations. 


In [70]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_4'] = results_4
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 5

In [71]:
# Product Similarity Score of all documents to Principle #5
results_5 = similarity_score(principle_number = 5)
#results_5

UNESCO AI Ethics Princple #5:
Privacy, a right essential to the protection of human dignity, human autonomy and human agency, must be respected, protected and promoted throughout the life cycle of AI systems. It is important that data for AI systems be collected, used, shared, archived and deleted in ways that are consistent with international law and in line with the values and principles set forth in this Recommendation, while respecting relevant national, regional and international legal frameworks.
Adequate data protection frameworks and governance mechanisms should be established in a multi-stakeholder approach at the national or international level, protected by judicial systems, and ensured throughout the life cycle of AI systems. Data protection frameworks and any related mechanisms should take reference from international data protection principles and standards concerning the collection, use and disclosure of personal data and exercise of their rights by data subjects while e

In [72]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_5'] = results_5
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 6

In [73]:
# Product Similarity Score of all documents to Principle #6
results_6 = similarity_score(principle_number = 6)
#results_6

UNESCO AI Ethics Princple #6:
Member States should ensure that it is always possible to attribute ethical and legal responsibility for any stage of the life cycle of AI systems, as well as in cases of remedy related to AI systems, to physical persons or to existing legal entities. Human oversight refers thus not only to individual human oversight, but to inclusive public oversight, as appropriate.
It may be the case that sometimes humans would choose to rely on AI systems for reasons of efficacy, but the decision to cede control in limited contexts remains that of humans, as humans can resort to AI systems in decision-making and acting, but an AI system can never replace ultimate human responsibility and accountability. As a rule, life and death decisions should not be ceded to AI systems.


In [74]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_6'] = results_6
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 7

In [75]:
# Product Similarity Score of all documents to Principle #7
results_7 = similarity_score(principle_number = 7)
#results_7

UNESCO AI Ethics Princple #7:
The transparency and explainability of AI systems are often essential preconditions to ensure the respect, protection and promotion of human rights, fundamental freedoms and ethical principles. Transparency is necessary for relevant national and international liability regimes to work effectively. A lack of transparency could also undermine the possibility of effectively challenging decisions based on outcomes produced by AI systems and may thereby infringe the right to a fair trial and effective remedy, and limits the areas in which these systems can be legally used.
While efforts need to be made to increase transparency and explainability of AI systems, including those with extra-territorial impact, throughout their life cycle to support democratic governance, the level of transparency and explainability should always be appropriate to the context and impact, as there may be a need to balance between transparency and explainability and other principles s

In [76]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_7'] = results_7
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 8

In [77]:
# Product Similarity Score of all documents to Principle #8
results_8 = similarity_score(principle_number = 8)
#results_8

UNESCO AI Ethics Princple #8:
AI actors and Member States should respect, protect and promote human rights and fundamental freedoms, and should also promote the protection of the environment and ecosystems, assuming their respective ethical and legal responsibility, in accordance with national and international law, in particular Member States’ human rights obligations, and ethical guidance throughout the life cycle of AI systems, including with respect to AI actors within their effective territory and control. The ethical responsibility and liability for the decisions and  actions based in any way on an AI system should always ultimately be attributable to AI actors corresponding to their role in the life cycle of the AI system.
Appropriate oversight, impact assessment, audit and due diligence mechanisms, including whistle-blowers’ protection, should be developed to ensure accountability for AI systems and their impact throughout their life cycle. Both technical and institutional desi

In [78]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_8'] = results_8
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 9

In [79]:
# Product Similarity Score of all documents to Principle #9
results_9 = similarity_score(principle_number = 9)
#results_9

UNESCO AI Ethics Princple #9:
Learning about the impact of AI systems should include learning about, through and for human rights and fundamental freedoms, meaning that the approach and understanding of AI systems should be grounded by their impact on human rights and access to rights, as well as on the environment and ecosystems


In [80]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_9'] = results_9
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

## Principle 10

In [81]:
# Product Similarity Score of all documents to Principle #10
results_10 = similarity_score(principle_number = 10)
#results_10

UNESCO AI Ethics Princple #10:
International law and national sovereignty must be respected in the use of data. That means that States, complying with international law, can regulate the data generated within or passing through their territories, and take measures towards effective regulation of data, including data protection, based on respect for the right to privacy in accordance with international law and other human rights norms and standards.
Participation of different stakeholders throughout the AI system life cycle is necessary for inclusive approaches to AI governance, enabling the benefits to be shared by all, and to contribute to sustainable development. Stakeholders include but are not limited to governments, intergovernmental organizations, the technical community, civil society, researchers and academia, media, education, policy-makers, private sector companies, human rights institutions and equality bodies, anti-discrimination monitoring bodies, and groups for youth and 

In [82]:
# Create a new column storing the results in the Dataset
similarity_data['Principle_10'] = results_10
similarity_data = similarity_data.loc[:,~similarity_data.T.duplicated(keep='first')]
#similarity_data.head()
#similarity_data.info()

# Step 5: Create Results Dataset

## Produce a CSV File

In [85]:
final_results = similarity_data.drop(columns = ['Main Text'])
final_results.info()
final_results.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company Name  49 non-null     object 
 1   Principle_1   49 non-null     float64
 2   Principle_2   49 non-null     float64
 3   Principle_3   49 non-null     float64
 4   Principle_4   49 non-null     float64
 5   Principle_5   49 non-null     float64
 6   Principle_6   49 non-null     float64
 7   Principle_7   49 non-null     float64
 8   Principle_8   49 non-null     float64
 9   Principle_9   49 non-null     float64
 10  Principle_10  49 non-null     float64
dtypes: float64(10), object(1)
memory usage: 4.3+ KB


,Company Name,Principle_1,Principle_2,Principle_3,Principle_4,Principle_5,Principle_6,Principle_7,Principle_8,Principle_9,Principle_10
0,Accenture,0.717751,0.702382,0.622116,0.619037,0.695432,0.667470,0.679528,0.698220,0.679119,0.609393
1,Adobe,0.566864,0.510722,0.576407,0.499825,0.604011,0.532677,0.571322,0.575822,0.624023,0.515835
2,Alphabet,0.630594,0.664262,0.607258,0.495027,0.723127,0.607318,0.684914,0.612065,0.668277,0.591854
3,Amazon,0.596675,0.681675,0.566090,0.501614,0.677715,0.510401,0.623684,0.591917,0.627909,0.596835
4,Atos,0.709803,0.718689,0.629426,0.643502,0.736860,0.654673,0.691443,0.722562,0.692313,0.635033


In [86]:
# Use this code to download the dataset
final_results.to_csv('Similarity_Analysis_Results.csv', index=False)

## Convert the similarity results to degree of representation of each principles

Note: Results are multiplied with 100 to indicate representation by percentage

In [103]:
principle_representation = final_results.set_index('Company Name')
principle_representation = principle_representation.div(principle_representation.sum(axis=1), axis=0).reset_index()
principle_representation = principle_representation[['Principle_1', 'Principle_2', 'Principle_3',
       'Principle_4', 'Principle_5', 'Principle_6', 'Principle_7',
       'Principle_8', 'Principle_9', 'Principle_10']] * 100
principle_representation.info()
principle_representation.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Principle_1   49 non-null     float64
 1   Principle_2   49 non-null     float64
 2   Principle_3   49 non-null     float64
 3   Principle_4   49 non-null     float64
 4   Principle_5   49 non-null     float64
 5   Principle_6   49 non-null     float64
 6   Principle_7   49 non-null     float64
 7   Principle_8   49 non-null     float64
 8   Principle_9   49 non-null     float64
 9   Principle_10  49 non-null     float64
dtypes: float64(10)
memory usage: 4.0 KB


,Principle_1,Principle_2,Principle_3,Principle_4,Principle_5,Principle_6,Principle_7,Principle_8,Principle_9,Principle_10
0,10.727994,10.498285,9.298569,9.252547,10.394396,9.976466,10.156683,10.436076,10.150572,9.108412
1,10.163394,9.156820,10.334488,8.961437,10.829400,9.550442,10.243328,10.323997,11.188201,9.248494
2,10.033808,10.569522,9.662492,7.876699,11.506161,9.663438,10.898119,9.738972,10.633402,9.417388
3,9.987002,11.409703,9.475079,8.395897,11.343433,8.542972,10.439073,9.907367,10.509789,9.989686
4,10.385886,10.515913,9.209805,9.415759,10.781785,9.579214,10.117238,10.572582,10.129966,9.291851


In [104]:
# Use this code to download the dataset
principle_representation.to_csv('Principle_Representation_Results.csv', index=False)